**Importing the dataset**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
data_yelp=pd.read_csv('yelp_labelled.txt',sep='\t',header=None)

In [ ]:
data_yelp.head()


,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
data_yelp.shape

(1000, 2)

In [ ]:
data_amazon=pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)

In [ ]:
data_amazon.head()


,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [ ]:
data_amazon.shape

(1000, 2)

In [ ]:
data_imdb=pd.read_csv('imdb_labelled.txt',sep='\t',header=None)

In [ ]:
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [ ]:
data_imdb.shape

(748, 2)

In [ ]:
data=data_yelp.append([data_amazon,data_imdb],ignore_index=True)

In [ ]:
data.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
data.shape

(2748, 2)

In [ ]:
data.columns=["Review","Sentiment"]


In [ ]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [ ]:
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [ ]:
x = data['Review']
y = data['Sentiment']

**Data Cleaning **

Remove punctatution,stop words and apply Lemmitization

In [ ]:
import string

In [ ]:
punct=string.punctuation

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:

from spacy.lang.en.stop_words import STOP_WORDS


In [ ]:
stop_words=list( STOP_WORDS)

In [ ]:
print(stop_words)

['front', 'out', 'its', 'show', '’ve', 'should', 'unless', 'became', 'nobody', 'well', 'doing', 'else', 'almost', 'many', 'did', 'except', 'hereafter', 'that', 'ever', 'am', 'fifteen', 'seeming', 'four', 'will', 'much', "n't", 'not', 'third', 'into', 'it', '‘s', 'elsewhere', 'then', 'throughout', 'up', 'has', 'thence', 'through', 'own', 'ten', 'yours', 'yet', 'all', 'i', 'one', 'last', 'would', 'rather', 'also', 'anywhere', 'n’t', 'both', 'have', 'call', 'on', 'keep', 'every', 'already', 'thru', 'cannot', 'six', '’s', 'besides', 'becomes', 'just', 'otherwise', 'they', '‘m', 'forty', 'neither', 'formerly', 'whole', 'anything', 'a', 'my', 'always', 'the', 'whenever', 'beforehand', 'therein', 'empty', 'and', 'nowhere', 'anyone', '‘ll', 'over', 'down', 'whoever', 'was', 'somehow', 'afterwards', 'everyone', 'thereafter', 'himself', 'latter', 'name', 'about', 'whereas', 'in', 'hereupon', 'everything', 'what', 'made', 'three', 'off', 'please', 'herself', 'had', 'nothing', 'this', 'less', 'don

In [ ]:
#function for data cleaning
def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
 
  cleaned_tokens = []
  for token in tokens:
    if token not in stop_words and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [ ]:
# if root form of that word is not pronoun then it is going to convert that into lower form
# and if that word is a proper noun, then we are directly taking lower form, because there is no lemma for proper noun

In [ ]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

Vectorization Feature Engineering (TF-IDF)

In [ ]:
import sklearn 
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)

In [ ]:

classifier = LinearSVC()

In [ ]:
#training the model

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state = 0)

In [ ]:
x_train.shape, x_test.shape

((2198,), (550,))

In [ ]:
x_train.head()

2572    An Italian reviewer called this "a small, grea...
526                          And it was way to expensive.
1509    As an earlier review noted, plug in this charg...
144     Nice blanket of moz over top but i feel like t...
2483    The film gives meaning to the phrase, "Never i...
Name: Review, dtype: object

Fit the x_train and y_train

In [ ]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])

In [ ]:
clf.fit(x_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7f86fee4ddd0>)),
                ('clf', LinearSVC())])

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
confusion_matrix(y_test, y_pred)

array([[201,  78],
       [ 50, 221]])

In [ ]:
# classification_report
print(classification_report(y_test, y_pred))
# we are getting almost 77% accuracy

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [ ]:
accuracy_score(y_test, y_pred)
# 76% accuracy

0.7672727272727272

In [ ]:
clf.predict(["Wow, I am learning Natural Language Processing in fun fashion!"])

array([1])

In [ ]:
clf.predict(["It's hard to learn new things!"])
# output is 0, that means review is Negative

array([0])

In [ ]:
import pickle
# create an iterator object with write permission - model.pkl
with open('sentimentanalysis_pkl', 'wb') as files:
    pickle.dump(clf, files)

In [ ]:
# load saved model
with open('sentimentanalysis_pkl' , 'rb') as f:
    lr = pickle.load(f)